# Secure SageMaker on AWS

In [1]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

## Internet Egress

In [ ]:
!curl https://aws.amazon.com/sagemaker/

## Data Protection

#### Check out the restrictive IAM policy attached to this user profile
```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": [
                "s3:GetObject",
                "s3:PutObject",
                "s3:ListObject"
            ],
            "Effect": "Allow",
            "Resource": [
                "arn:aws:s3:::sagemaker-us-east-1-835319576252-secure",
                "arn:aws:s3:::sagemaker-us-east-1-835319576252-secure/*"
            ]
        }
    ]
}
```

### Let's try to copy data over to a different S3 bucket!

In [3]:
!echo s3://$bucket-secure/

s3://sagemaker-us-east-1-835319576252-secure


In [ ]:
!aws s3 cp security.ipynb s3://$bucket-secure/

### Let's try to copy data over to the allowed S3 bucket!

In [ ]:
!aws s3 cp ./security.ipynb s3://$bucket-secure/

### Let's test permissions again with a bucket policy and a VPC endpoint policy attached!

### Bucket policy
```
{
    "Version": "2008-10-17",
    "Statement": [
        {
            "Effect": "Deny",
            "Principal": "*",
            "Action": [
                "s3:GetObject",
                "s3:PutObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::sagemaker-us-east-1-835319576252-secure",
                "arn:aws:s3:::sagemaker-us-east-1-835319576252-secure/*"
            ],
            "Condition": {
                "StringNotEquals": {
                    "aws:sourceVpce": "vpce-<ADD_VPC_ID_HERE>"
                }
            }
        }
    ]
}
```

### VPC endpoint policy: -- notice changed bucket name
```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": "*",
            "Action": [
                "s3:GetObject",
                "s3:PutObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::sagemaker-us-east-1-835319576252-secure-diff-name",
                "arn:aws:s3:::sagemaker-us-east-1-835319576252-secure-diff-name/*"
            ]
        }
    ]
}
```

In [7]:
# Run this cell in a terminal
# TODO:  Not sure what this cell is supposed to do

!aws s3 cp s3://$bucket-secure/ .

fatal error: An error occurred (NoSuchBucket) when calling the ListObjectsV2 operation: The specified bucket does not exist


## Train without VPC mode

Let's kick off a training job without VPC mode enabled and see what happens

In [9]:
# %%writefile mnist.py

# import tensorflow as tf
# import argparse
# import os
# import numpy as np

# def parse_args():
    
#     parser = argparse.ArgumentParser()
#     parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
#     parser.add_argument('--test', type=str, default=os.environ.get('SM_CHANNEL_TEST'))
    
#     parser.add_argument('--model_dir', type=str, default=os.environ.get('SM_MODEL_DIR'))
    
#     return parser.parse_known_args()

# def get_train_data(train_dir):
    
#     x_train = np.load(os.path.join(train_dir, 'x_train.npy'))
#     y_train = np.load(os.path.join(train_dir, 'y_train.npy'))
#     print('x train', x_train.shape,'y train', y_train.shape)

#     return x_train, y_train


# if __name__ == "__main__":
    
#     args, _ = parse_args()
    
#     x_train, y_train = get_train_data(args.train)
    
    
#     model = tf.keras.models.Sequential([
#             tf.keras.layers.Flatten(input_shape=(28, 28)),
#             tf.keras.layers.Dense(128, activation='relu'),
#             tf.keras.layers.Dropout(0.2),
#             tf.keras.layers.Dense(10, activation='softmax')
#     ])

#     model.compile(optimizer='adam',
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])

#     model.fit(x_train, y_train, epochs=1)

#     print("Training Complete")


Writing mnist.py


In [ ]:
!aws s3 cp x_train.npy s3://$bucket/train/
!aws s3 cp y_train.npy s3://$bucket/train/

In [ ]:
import sagemaker
sess = sagemaker.Session()

train_s3 = "s3://{}/train/".format(bucket)
print(train_s3)
inputs = {'train':train_s3}

print(inputs)

## Set IAM Policy to Allow Training Only In Specific VPCs/SecurityGroups/SubnetIds
```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VPCDeployment",
            "Effect": "Deny",
            "Action": [
                "sagemaker:CreateAutoMLJob",
                "sagemaker:CreateTrainingJob",
                "sagemaker:CreateProcessingJob",
                "sagemaker:CreateModel",
                "sagemaker:CreateHyperParameterTuningJob"
            ],
            "Resource": "*",
            "Condition": {
                "StringNotEquals": {
                    "sagemaker:VpcSecurityGroupIds": "<ADD_SECURITY_GROUP_IDS_HERE>",
                    "sagemaker:VpcSubnets": [
                        "subnet-<ADD_SUBNET_IDS_HERE>",
                        "subnet-<ADD_SUBNET_IDS_HERE>"
                    ]
                }
            }
        }
    ]
}
```

In [ ]:
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow

role = get_execution_role()
print(role)
mnist_estimator = TensorFlow(entry_point='mnist.py',
                             role=role,
                             train_instance_count=1,
                             train_instance_type='ml.m5.xlarge',
                             framework_version='1.15.2',
                             py_version='py3',
                             output_path = 's3://{}/output/'.format(bucket))

mnist_estimator.fit(inputs)

## Train with VPC mode

Let's kick off a training job with VPC mode enabled and make sure that it goes through!

In [6]:
# !pip install sagemaker_environment

ERROR: Could not find a version that satisfies the requirement sagemaker_environment
ERROR: No matching distribution found for sagemaker_environment


In [4]:
# import sagemaker_environment
subnet = sagemaker_environment.SAGEMAKER_SUBNETS
security_group_ids = sagemaker_environment.SAGEMAKER_SECURITY_GROUPS
key = sagemaker_environment.CMK_ID
print(key)

ModuleNotFoundError: No module named 'sagemaker_environment'

In [ ]:
from sagemaker import get_execution_role
role = get_execution_role()
print(role)
mnist_estimator = TensorFlow(entry_point='mnist.py',
                             role=role,
                             train_instance_count=1,
                             train_instance_type='ml.m5.xlarge',
                             framework_version='1.15.2',
                             py_version='py3',
                             output_path='s3://{}/output/'.format(bucket),
                             subnets=subnet,
                             security_group_ids=security_group_ids,
                             output_kms_key=key)

mnist_estimator.fit(inputs)

## Train without encryption

Let's kick off a training job without encryption and check if the SCP kicks in!

In [ ]:
from sagemaker import get_execution_role
role = get_execution_role()
print(role)
mnist_estimator = TensorFlow(entry_point='mnist.py',
                             role=role,
                             train_instance_count=1,
                             train_instance_type='ml.m5.xlarge',
                             framework_version='1.15.2',
                             py_version='py3',
                             output_path = 's3://{}/output/'.format(bucket),
                             subnets = subnet,
                             security_group_ids = security_group_ids)

mnist_estimator.fit(inputs)